# BERT 전이학습

### 데이터 준비

In [15]:
from tensorflow.keras.utils import get_file

# NSMC (네이버 영화리뷰) 데이터 다운로드
ratings_train_path = get_file('ratings_train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
ratings_test_path = get_file('ratings_test.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

In [16]:
import pandas as pd

ratings_train_df = pd.read_csv(ratings_train_path, sep='\t')  # 학습 데이터 로드
ratings_test_df = pd.read_csv(ratings_test_path, sep='\t')

display(ratings_train_df.head())  # 상위 5개 출력
display(ratings_test_df.head())

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [17]:
ratings_train_df = ratings_train_df.dropna(how='any')  # 학습 데이터 결측치 포함 행 제거
ratings_test_df = ratings_test_df.dropna(how='any')

display(ratings_train_df.info)
display(ratings_test_df.info)

<bound method DataFrame.info of               id                                           document  label
0        9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1        3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2       10265843                                  너무재밓었다그래서보는것을추천한다      0
3        9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4        6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
...          ...                                                ...    ...
149995   6222902                                인간이 문제지.. 소는 뭔죄인가..      0
149996   8549745                                      평점이 너무 낮아서...      1
149997   9311800                    이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?      0
149998   2376369                        청춘 영화의 최고봉.방황과 우울했던 날들의 자화상      1
149999   9619869                           한국 영화 최초로 수간하는 내용이 담긴 영화      0

[149995 rows x 3 columns]>

<bound method DataFrame.info of             id                                           document  label
0      6270596                                                굳 ㅋ      1
1      9274899                               GDNTOPCLASSINTHECLUB      0
2      8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3      6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4      6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0
...        ...                                                ...    ...
49995  4608761          오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함      1
49996  5308387       의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO      0
49997  9072549                 그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다      0
49998  5802125     절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네      0
49999  6070594                                         마무리는 또 왜이래      0

[49997 rows x 3 columns]>

In [18]:
ratings_train_df = ratings_train_df.sample(n=15000, random_state=0)  # 학습 데이터 15000개 무작위 추출 (재현성 고정)
ratings_test_df = ratings_test_df.sample(n=5000, random_state=0)

# 라벨 (0/1) 개수 분포 확인
ratings_train_df['label'].value_counts(), ratings_test_df['label'].value_counts()

(label
 0    7512
 1    7488
 Name: count, dtype: int64,
 label
 0    2532
 1    2468
 Name: count, dtype: int64)

In [19]:
# 텍스트/라벨을 리스트로 변환 (학습/테스트 분리)
X_train = ratings_train_df['document'].values.tolist()  # 학습 입력 문장(document) 컬럼을 리스트로 변환
y_train = ratings_train_df['label'].values.tolist()

X_test = ratings_test_df['document'].values.tolist()    # 테스트 입력 문장(document) 컬럼을 리스트로 변환
y_test = ratings_test_df['label'].values.tolist()

len(X_train), len(y_train), len(X_test), len(y_test)

(15000, 15000, 5000, 5000)

### 토크나이저/모델 준비

- bert 한국어 버전 사전학습 모델 klue/bert-base

In [20]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('klue/bert-base')          # 사전학습 KLUE BERT 모델 로드 (가중치 포함)
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')  # 사전학습 KLUE BERT 토크나이저 로드 (토큰화 규칙/어휘)

In [ ]:
tokenizer.model_max_length  # 512 토큰을 넘으면 토큰화시 자름

512

In [ ]:
X_train = tokenizer(X_train, padding=True, truncation=True)  # 가장 긴 문장을 가진 토큰 기준으로 맞춰진다.
X_test = tokenizer(X_test, padding=True, truncation=True)

X_train[:5], X_test[:5]

([Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])],
 [Encoding(num_tokens=118, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=118, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=118, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_

In [ ]:
print(X_train['input_ids'][0])        # 토큰 -> 정수ID로 변환된 시퀀스
print(X_train['attention_mask'][0])   # 실제 토큰=1, 패딩=0으로 구분한 마스크
print(X_train['token_type_ids'][0])   # 문장 구분 ID (단일 문장은 모두 0)

[2, 1800, 2178, 860, 3629, 16516, 2031, 18, 18, 18, 14242, 2205, 2062, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### 데이터 파이프라인 생성

In [ ]:
type(X_train)    # tokenizer 반환 객체 타입

transformers.tokenization_utils_base.BatchEncoding

In [30]:
# Tensorflow 의 tf.datea.Dataset 생성 (BERT 입력 + 라벨 묶기)
import tensorflow as tf

train_ds = tf.data.Dataset.from_tensor_slices((dict(X_train), y_train))  # 토큰화 dict + 라벨 = Dataset 변환 (학습용)
test_ds = tf.data.Dataset.from_tensor_slices((dict(X_test), y_test))     # 토큰화 dict + 라벨 = Dataset 변환 (테스트용)

In [31]:
# 학습 : 셔플 -> 미니배치 -> 프리배치로 파이프라인 최적화
train_dataset = train_ds.shuffle(10000).batch(64).prefetch(tf.data.AUTOTUNE)
# 테스트 : 미니배치 -> 프리배치로 파이프라인 최적화
test_dataset = test_ds.batch(64).prefetch(tf.data.AUTOTUNE)

.prefetch(tf.data.AUTOTUNE) : 계산이랑 데이터 준비를 겹쳐서 미리 준비해둔다. (적당한 갯수는 Tensorflow에서 알아서 계산한다.)  
-> [데이터 로딩 + 모델 계산] -> [데이터 로딩 + 모델 계산]

In [33]:
%pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 18.1 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [34]:
# KLUE BERT 분류 모델 로드
from transformers import TFBertForSequenceClassification

# 2클래스 분류용 TF BERT 로드 (Pytorch 가중치 변환)
model = TFBertForSequenceClassification.from_pretrained('klue/bert-base', num_labels=2, from_pt=True)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should p

In [ ]:
# BERT 파인튜닝 설정 및 학습 (옵티마이저/컴파일/학습)
from transformers import create_optimizer

num_train_steps = len(train_dataset) * 5       # 전체확습 step 수(steps_per_epoch * 5(epochs))
num_warmup_steps = int(num_train_steps * 0.1)  # warmup step 수(전체의 10%)

# Transformers 권장 스케줄 포함 옵티마이저 생성
optimizer, _ = create_optimizer(
    init_lr = 5e-5,
    num_train_steps = num_train_steps,        # 총 학습 step 수
    num_warmup_steps = num_warmup_steps,      # warmup step 수
    weight_decay_rate = 0.1                   # weight decay 비율 (가중치 감쇠: 과적합 완화)
)

# 모델 학습설정
model.compile(
    optimizer = optimizer,
    loss = model.hf_compute_loss,    # HF 제공 loss 계산 함수
    metrics = ['accuracy']
)

# 모델 학습
model.fit(train_dataset, epochs=5,
            validation_data = test_dataset,
            batch_size=64
)

Epoch 1/5


235/235 [==============================] - 4430s 19s/step - loss: 0.3935 - accuracy: 0.8172 - val_loss: 0.3055 - val_accuracy: 0.8734
Epoch 2/5
 68/235 [=======>......................] - ETA: 49:01 - loss: 0.2269 - accuracy: 0.9108

- 워밍업(warmup) 스텝이란 학습 초반에는 학습률을 바로 크게 쓰지않고, 작은 값에서 시작해서 일정 스텝동안 점진적으로 올리는 구간
    - 스텝 * 0.1 => 전체 스텝에서 10%동안 LR를 천천히 올리는 설정
    - 동작 과정 : num_warmup_steps 동안 LR를 0 -> init_lr로 선형으로 증가시키고, 워밍업이 끝나면 LR가 점점 감소한다.
    - 사용 이유 : BERT 같은 사전학습 모델이 파인튜닝시에 초반에 LR가 크면 가중치가 갑자기 크게 바뀌어서 학습이 불안정/발산이 일어나 성능이 흔들릴 가능성이 있다.

In [ ]:
# 파인튜닝 모델/토크나이저 저장
model.save_pretrained('nsmc_model/bert-base')        # 학습된 모델 가중치/설정(config) 저장
tokenizer.save_pretrained('nsmc_model/bert_base')    # 토크나이저 파일(vocab, tokenizer_config 등) 저장

### 추론

In [ ]:
# 텍스트 분류 라벨 값 지정
model.config.id2label = {
    0: "부정",
    1: "긍정"
}

In [ ]:
# 감성분석 파이프라인 생성
from transformers import TextClassificationPipeline

# 입력 텍스트 -> 토큰화 -> 모델 추론 -> 라벨/점수 반환 파이프라인
sentiment_classifier = TextClassificationPipeline(
    tokenizer = tokenizer,      # 사용할 토크나이저
    model = model,              # 사용할 분류 모델
    framework = 'tf',           # TensorFlow 기반 모델 사용
    return_all_scores = True    # 모든 라벨 점수(확률/스코어) 반환
)

In [ ]:
sentiment_classifier("인생 영화를 찾았습니다!!!")  # 입력 문장에 대해 부정/긍정 점수 예측

# HuggingFace

### push